In [83]:
import csv
import re
from enum import Enum
from pathlib import Path

import polars as pl
from polars import selectors as cs

In [ ]:
# dev: for POG1298

DATA_PATH = Path("../data/")

In [181]:
def count_vcf_metadata_rows(file_path: Path) -> int:
    with open(file_path) as infile:
        num_metadata_lines = 0
        while True:
            line = infile.readline()
            
            if re.match("##", line):
                num_metadata_lines += 1 
            else:
                break
    
    return num_metadata_lines

In [186]:
df = pl.read_csv(
    str(DATA_PATH / "sample.indels.vcf"),
    separator="\t",
    skip_lines=count_vcf_metadata_rows(DATA_PATH / "sample.indels.vcf"),
)

df.head()

#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NORMAL,TUMOR,F111183,F111704
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""chr1""",2081028,""".""","""TG""","""T""",""".""","""PASS""","""SOMATIC;QSI=75;TQSI=1;NT=ref;Q…","""GT:DP:DP2:TAR:TIR:TOR:DP50:FDP…",""".:54:54:53,54:0,0:1,1:50.50:0.…",""".:118:118:90,92:21,22:8,8:120.…","""0|0:54:.:.:.:.:.:.:.:.:54,0:0.…","""0|1:122:.:.:.:.:.:.:.:.:98,24:…"
"""chr1""",4031552,"""rs113283042;rs140543974;rs5768…","""G""","""GT""",""".""","""PASS""","""SOMATIC;QSI=59;TQSI=1;NT=ref;Q…","""DP:DP2:TAR:TIR:TOR:DP50:FDP50:…","""45:45:42,42:0,0:3,3:44.36:0.25…","""115:115:81,82:21,22:11,10:111.…",""".""","""."""
"""chr1""",6481562,""".""","""A""","""AATAT""",""".""","""PASS""","""SOMATIC;QSI=51;TQSI=1;NT=ref;Q…","""DP:DP2:TAR:TIR:TOR:DP50:FDP50:…","""43:43:36,38:3,3:4,3:44.67:1.32…","""111:111:57,62:36,39:21,19:110.…",""".""","""."""
"""chr1""",7156275,""".""","""CA""","""C""",""".""","""PASS""","""SOMATIC;QSI=59;TQSI=1;NT=ref;Q…","""DP:DP2:TAR:TIR:TOR:DP50:FDP50:…","""39:39:28,31:0,0:12,9:39.78:2.4…","""103:103:52,54:25,25:26,25:103.…",""".""","""."""
"""chr1""",11633506,"""rs573016490""","""AT""","""A""",""".""","""PASS""","""SOMATIC;QSI=61;TQSI=2;NT=ref;Q…","""GT:DP:DP2:TAR:TIR:TOR:DP50:FDP…",""".:34:34:28,28:0,0:5,5:31.28:1.…",""".:109:109:56,58:30,30:21,23:10…","""0/0:24:.:.:.:.:.:.:.:.:24,0:0.…","""0/1:70:.:.:.:.:.:.:.:.:42,28:0…"


In [38]:
df.columns

['#CHROM',
 'POS',
 'ID',
 'REF',
 'ALT',
 'QUAL',
 'FILTER',
 'INFO',
 'FORMAT',
 'NORMAL',
 'TUMOR',
 'F111183',
 'F111704']

In [109]:
genotype_format = """##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Read depth for tier1">
##FORMAT=<ID=DP2,Number=1,Type=Integer,Description="Read depth for tier2">
##FORMAT=<ID=TAR,Number=2,Type=Integer,Description="Reads strongly supporting alternate allele for tiers 1,2">
##FORMAT=<ID=TIR,Number=2,Type=Integer,Description="Reads strongly supporting indel allele for tiers 1,2">
##FORMAT=<ID=TOR,Number=2,Type=Integer,Description="Other reads (weak support or insufficient indel breakpoint overlap) for tiers 1,2">
##FORMAT=<ID=DP50,Number=1,Type=Float,Description="Average tier1 read depth within 50 bases">
##FORMAT=<ID=FDP50,Number=1,Type=Float,Description="Average tier1 number of basecalls filtered from original read depth within 50 bases">
##FORMAT=<ID=SUBDP50,Number=1,Type=Float,Description="Average number of reads below tier1 mapping quality threshold aligned across sites within 50 bases">
##FORMAT=<ID=BCN50,Number=1,Type=Float,Description="Fraction of filtered reads within 50 bases of the indel.">
##FORMAT=<ID=AD,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed">
##FORMAT=<ID=AF,Number=A,Type=Float,Description="Allele fractions of alternate alleles in the tumor">
##FORMAT=<ID=F1R2,Number=R,Type=Integer,Description="Count of reads in F1R2 pair orientation supporting each allele">
##FORMAT=<ID=F2R1,Number=R,Type=Integer,Description="Count of reads in F2R1 pair orientation supporting each allele">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=PGT,Number=1,Type=String,Description="Physical phasing haplotype information, describing how the alternate alleles are phased in relation to one another; will always be heterozygous and is not intended to describe called alleles">
##FORMAT=<ID=PID,Number=1,Type=String,Description="Physical phasing ID information, where each unique ID within a given sample (but not across samples) connects records within a phasing group">
##FORMAT=<ID=PL,Number=G,Type=Integer,Description="Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification">
##FORMAT=<ID=PS,Number=1,Type=Integer,Description="Phasing set (typically the position of the first variant in the set)">
##FORMAT=<ID=SB,Number=4,Type=Integer,Description="Per-sample component statistics which comprise the Fisher's Exact Test to detect strand bias.">"""

genotype_format_dict = {}

for line in genotype_format.split("\n"):
    line = re.sub(
        r"^##FORMAT=<|>$",
        "",
        line
    )
    try:
        values = re.split(
            r',(?=(?:[^"]*"[^"]*")*[^"]*$)',
            line                      
        )

        for value in values:
            if value.startswith("ID="):
                id = value
            elif value.startswith("Number="):
                num_values = value
            elif value.startswith("Type="):
                data_type = value
            elif value.startswith("Description="):
                description = value

        genotype_format_dict[id.split("=")[1]] = {
            "Number": num_values.split("=")[1],
            "Type": data_type.split("=")[1],
            "Description": description.split("=")[1].strip('"')
        }
    except ValueError:
        raise ValueError(f"FORMAT metadata is not well-formed for line {line}")

In [ ]:
df.select(
    pl.col("FORMAT", "TUMOR")
).filter(
    pl.col("FORMAT").str.split(":").list.len() != pl.col("TUMOR").str.split(":").list.len()
).unique(
    subset=["TUMOR"]    
).filter(
    pl.col("TUMOR").str.split(":").list.len() < 10
)

FORMAT,TUMOR
str,str
"""GT:AD:AF:DP:F1R2:F2R1:SB""","""."""


In [190]:
df.filter(
    pl.col("FORMAT").str.contains("PS")
).select(
    pl.col("FORMAT", "NORMAL", "TUMOR", "F111183", "F111704")
).row(0)

('GT:DP:DP2:TAR:TIR:TOR:DP50:FDP50:SUBDP50:BCN50:AD:AF:F1R2:F2R1:PGT:PID:PS:SB',
 '.:54:54:53,54:0,0:1,1:50.50:0.29:0.00:0.00',
 '.:118:118:90,92:21,22:8,8:120.73:0.00:0.00:0.00',
 '0|0:54:.:.:.:.:.:.:.:.:54,0:0.018:25,0:25,0:0|1:2081028_TG_T:2081028:25,29,0,0',
 '0|1:122:.:.:.:.:.:.:.:.:98,24:0.202:49,12:45,12:0|1:2081028_TG_T:2081028:46,52,10,14')

In [189]:
df.columns

['#CHROM',
 'POS',
 'ID',
 'REF',
 'ALT',
 'QUAL',
 'FILTER',
 'INFO',
 'FORMAT',
 'NORMAL',
 'TUMOR',
 'F111183',
 'F111704']

In [184]:
df = pl.read_csv(
    str(DATA_PATH / "sample.snvs.vcf"),
    separator="\t",
    skip_lines=count_vcf_metadata_rows(DATA_PATH / "sample.snvs.vcf"),
    has_header=True,
)

df.head()

#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,F111183,F111704
str,i64,str,str,str,str,str,str,str,str,str
"""chr1""",630074,"""rs7349151""","""T""","""C""",""".""","""PASS""","""AS_FilterStatus=SITE;AS_SB_TAB…","""GT:AD:AF:DP:F1R2:F2R1:PGT:PID:…","""0|0:51,0:0.019:51:18,0:22,0:0|…","""0|1:115,18:0.132:133:33,10:64,…"
"""chr1""",1395944,""".""","""A""","""C""",""".""","""PASS""","""AS_FilterStatus=SITE;AS_SB_TAB…","""GT:AD:AF:DP:F1R2:F2R1:SB""","""0/0:48,0:0.020:48:21,0:25,0:23…","""0/1:98,14:0.125:112:49,6:46,8:…"
"""chr1""",1415696,""".""","""C""","""A""",""".""","""PASS""","""AS_FilterStatus=SITE;AS_SB_TAB…","""GT:AD:AF:DP:F1R2:F2R1:SB""","""0/0:59,0:0.017:59:31,0:28,0:28…","""0/1:106,30:0.220:136:57,18:49,…"
"""chr1""",1777972,""".""","""G""","""A""",""".""","""PASS""","""AS_FilterStatus=SITE;AS_SB_TAB…","""GT:AD:AF:DP:F1R2:F2R1:SB""","""0/0:44,0:0.022:44:13,0:31,0:28…","""0/1:89,12:0.129:101:39,7:49,5:…"
"""chr1""",1959772,"""rs1015196821""","""C""","""T""",""".""","""PASS""","""AS_FilterStatus=SITE;AS_SB_TAB…","""GT:AD:AF:DP:F1R2:F2R1:SB""","""0/0:55,0:0.018:55:16,0:39,0:25…","""0/1:73,76:0.505:149:36,36:37,3…"
